In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import model_from_json

In [18]:
# 사용자 입력
station_name = '남위례'
date = '2025-01-01'
time = '12:00'

# 사용자 입력 처리 및 전처리
data = pd.read_csv(f'{station_name}2.csv', header=None)
data = data.values.astype('float32')
data = data.reshape(len(data), 1)

scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# 처음 데이터를 멜팅하기전에 이런식으로 배치되어있었음, 그래서 lookback을 설정해서 예측하고 싶은 시간대를 설정 가능
# 06	0607	0708	0809	0910	...	1415	1516	1617	1718	1819	1920	2021	2122	2223	23
# ~06 = 1, 06~07 = 2, 07~08 = 3 08~09 = 4, 09~10 = 5, 10~11 = 6, 11~12 = 7, 12~13 = 8, 13~14 = 9, 14~15 = 10, 15~16 = 11
# 16~17 = 12, 17~18 = 13, 18~19 = 14, 19~20 = 15, 20~21 = 16, 21~22 = 17, 22~23 = 18, 23~ = 19
# 예를 들어, 만약 19개의 시간대를 사용하고 있다면 18번째 시간대의 데이터를 기반으로 다음 시간대(19번째 시간대)를 예측함
look_back = 19

In [19]:
#모델 로드
json_file = open('model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('model2_weights.h5')

# 입력 데이터 생성
input_data = data[-look_back:]

# 모델 예측
input_data = np.reshape(input_data, (1, look_back, 1)) # 입력데이터 3d 형태로 변환 
prediction = model.predict(input_data)

# 예측 결과 출력
scaled_prediction = scaler.inverse_transform(np.array([[prediction[0, 0]]])) # 실제 값으로 변환
prediction_percent = (scaled_prediction[0, 0] / 26541) * 250
print(f"예측된 혼잡도: {prediction_percent:.2f}%")

1/1 [==============================] - 2s 2s/step
예측된 혼잡도: 112.37%
